In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
/kaggle/input/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
/kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py


In [2]:
from lightgbm import LGBMRegressor
import pandas as pd
from tqdm import tqdm
import numpy as np

In [3]:
train_path = "/kaggle/input/optiver-trading-at-the-close/train.csv"


def get_x(df: pd.DataFrame, fill_nan=1):
    x = df[
        [
            "imbalance_size",
            "imbalance_buy_sell_flag",
            "reference_price",
            "matched_size",
            "far_price",
            "near_price",
            "bid_price",
            "bid_size",
            "ask_price",
            "ask_size",
            "wap",
        ]
    ]
    x["is/ms"] = x["imbalance_size"] / x["matched_size"]
    x["is/bs"] = x["imbalance_size"] / x["bid_size"]
    x["is/as"] = x["imbalance_size"] / x["ask_size"]
    x["bs/as"] = x["bid_size"] / x["ask_size"]
    x["fp/np"] = x["far_price"] / x["near_price"]
    x["bp/ap"] = x["bid_price"] / x["ask_price"]
    x = x[
        [
            "imbalance_buy_sell_flag",
            "reference_price",
            "far_price",
            "near_price",
            "bid_price",
            "ask_price",
            "wap",
            "is/ms",
            "is/bs",
            "is/as",
            "bs/as",
            "fp/np",
            "bp/ap",
        ]
    ]
    x = x.values.astype(np.float32)
    x = np.nan_to_num(x, fill_nan)
    return x


def get_y(df: pd.DataFrame):
    y = df["target"]
    y = y.values.astype(np.float32)
    return y


data = pd.read_csv(train_path)

x = get_x(data)
y = get_y(data)

/tmp/ipykernel_19/1898295309.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["is/ms"] = x["imbalance_size"] / x["matched_size"]
/tmp/ipykernel_19/1898295309.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["is/bs"] = x["imbalance_size"] / x["bid_size"]
/tmp/ipykernel_19/1898295309.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

In [4]:
# remove rows having NaN target
nan_rows = np.isnan(y)
x = x[~nan_rows]
y = y[~nan_rows]

train_ratio = 0.8
train_val_split_idx = int(len(x) * train_ratio)

x_train = x[:train_val_split_idx]
y_train = y[:train_val_split_idx]
x_val = x[train_val_split_idx:]
y_val = y[train_val_split_idx:]

del data

In [5]:
seed0 = 123
params = {
    # 'early_stopping_rounds': 50,
    "objective": "regression",
    "metric": "rmse",
    #     'metric': 'None',
    "boosting_type": "gbdt",
    "max_depth": 20,
    "verbose": -1,
    "max_bin": 300,
    "min_data_in_leaf": 50,
    "learning_rate": 0.03,
    "subsample": 0.7,
    "subsample_freq": 1,
    "feature_fraction": 1,
    "lambda_l1": 0.5,
    "lambda_l2": 2,
    "seed": seed0,
    "feature_fraction_seed": seed0,
    "bagging_fraction_seed": seed0,
    "drop_seed": seed0,
    "data_random_seed": seed0,
    "extra_trees": True,
    "extra_seed": seed0,
    "zero_as_missing": True,
    "first_metric_only": True,
}

model = LGBMRegressor(**params)
model.fit(x_train, y_train, eval_set=[(x_val, y_val)])

del x_train, y_train
model

[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=2, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[1]	valid_0's rmse: 8.99783
[2]	valid_0's rmse: 8.99484
[3]	valid_0's rmse: 8.99147
[4]	valid_0's rmse: 8.98811
[5]	valid_0's rmse: 8.98506
[6]	valid_0's rmse: 8.98201
[7]	valid_0's rmse: 8.97926
[8]	valid_0's rmse: 8.9762
[9]	valid_0's rmse: 8.97351
[10]	valid_0's rmse: 8.97104
[11]	valid_0's rmse: 8.96865
[12]	valid_0's rmse: 8.96615
[13]	valid_0's rmse: 8.964
[14]	valid_0's rmse: 8.96221
[15]	valid_0's rmse: 8.9601
[16]	valid_0's rmse: 8.95804
[17]	valid_0's rmse: 8.95642
[18]	valid_0's rmse: 8.95453
[19]	valid_0's rmse: 8.95292
[20]	valid_0's rm

LGBMRegressor(bagging_fraction_seed=123, data_random_seed=123, drop_seed=123,
              extra_seed=123, extra_trees=True, feature_fraction=1,
              feature_fraction_seed=123, first_metric_only=True, lambda_l1=0.5,
              lambda_l2=2, learning_rate=0.03, max_bin=300, max_depth=20,
              metric='rmse', min_data_in_leaf=50, objective='regression',
              seed=123, subsample=0.7, subsample_freq=1, verbose=-1,
              zero_as_missing=True)

In [6]:
# evaluate
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

yhat = model.predict(x_val)

mae = mean_absolute_error(yhat, y_val)
print(f'Mean Absolute Error (MAE): {mae}')

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(yhat, y_val)
print(f'Mean Squared Error (MSE): {mse}')

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse}')


mae = np.mean(np.abs(yhat - y_val))
print("MAE", mae)

Mean Absolute Error (MAE): 5.984826398636267
Mean Squared Error (MSE): 79.48395856416758
Root Mean Squared Error (RMSE): 8.915377645628231
MAE 5.984826398636267


In [7]:
# make submission
import optiver2023

env = optiver2023.make_env()
iter_test = env.iter_test()
counter = 0
for test, revealed_targets, sample_prediction in tqdm(iter_test):
    x_test = get_x(test)
    y_hat = model.predict(x_test)
    sample_prediction["target"] = y_hat
    env.predict(sample_prediction)
    counter += 1

0it [00:00, ?it/s]

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


/tmp/ipykernel_19/1898295309.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["is/ms"] = x["imbalance_size"] / x["matched_size"]
/tmp/ipykernel_19/1898295309.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["is/bs"] = x["imbalance_size"] / x["bid_size"]
/tmp/ipykernel_19/1898295309.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o